Recitation 1: Encryption
========================

In this lab we will get hands-on experience with different encryption methods: The Caesar Cipher, AES, and RSA.

Start by running the command below, to install the required libraries.

In [ ]:
!pip install pycrypto

First, let's come up with a plaintext string which we will encrypt. Imagine this is some text we want to keep secret. Feel free to replace it with something else, but this exercise works best when it's about a paragraph long:

In [ ]:
plaintext = (
    "The FitnessGram Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues. "
    "The 20 meter pacer test will begin in 30 seconds. Line up at the start. The running speed starts slowly, but gets faster each minute after you hear this signal. "
    "A single lap should be completed each time you hear this sound. Remember to run in a straight line, and run as long as possible.")

Part 1: Caesar Cipher
---------------------
We would like to encrypt this message so that only we can read it. Let's start off by using a caesar cypher: this is a simple method of scrambling a message, by shifting each character by a secret amount. So if the secret were three, "a" would shift three letters to become "d", "h" would become "k", and so on. We can define a function to do this given a plaintext string and a secret shift value:

In [ ]:
import string

def caesar_cipher(plaintext: str, secret: int):
    # This function shifts a single character.
    def shift_char(char: str):
        # Ignore special characters.
        if char not in string.ascii_letters:
            return char

        # We need to do a little math so that letters at the end of the alphabet wrap around.
        if char.isupper():
            return chr((ord(char) - ord("A") + secret) % 26 + ord("A"))
        else:
            return chr((ord(char) - ord("a") + secret) % 26 + ord("a"))
    
    # Shift each character by the secret amount
    shifted_chars = [shift_char(char) for char in plaintext]

    # Join them back into a string
    return "".join(shifted_chars)

Now when we run the code below, we can see that we've scrambled the text, and nobody can read it without the secret key! ...right?

In [ ]:
secret_key = 7

ciphertext = caesar_cipher(plaintext, secret_key)

print("Ciphertext:")
print("===========")
print(ciphertext)

We can reverse the process by shifting backwards by the same amount. Only the same secret will work however, and shifting by a different amount will still result in scrambled text:

In [ ]:
print("Plaintext:")
print("==========")
print(caesar_cipher(ciphertext, -secret_key))
print()
print("Incorrect plaintext (uses the wrong secret key):")
print("==========")
print(caesar_cipher(ciphertext, -secret_key + 1))

Part 1a: Brute Forcing
---------------------
So far, this cipher can scramble a message so it cannot be read, and it can only be unscrambled with the secret key. We're secure now, right? Well, not really. For one thing, there's only 25 possible shifts you can make before the characters loop back around. An attacker could simply try each one and see which one is correct. Below is some unknown ciphertext, see if you can figure out what it is just by iterating through all 25 shifts:

In [ ]:
unknown_ciphertext = "Aipp, xlex aew vieppc iewc xs fvieo."
print(caesar_cipher(unknown_ciphertext, -1))

Part 1b: Frequency Analysis
--------------------------

The small number of possiblities isn't the only problem, however. The Caesar Cipher does not scramble the characters very much, and there are a lot of patterns that we can use to determine the secret key. One tactic is frequency analysis: looking at which letters show up most often. The English language uses some letters more than others, so letters like "E", "A", and "T" are very common while letters like "Z", "X", and "J" are almost never used. We can find these frequencies online, and graph the result:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

letter_frequencies = {
    'A': 8.167,
    'B': 1.492,
    'C': 2.782,
    'D': 4.253,
    'E': 12.702,
    'F': 2.228,
    'G': 2.015,
    'H': 6.094,
    'I': 6.996,
    'J': 0.153,
    'K': 0.772,
    'L': 4.025,
    'M': 2.406,
    'N': 6.749,
    'O': 7.507,
    'P': 1.929,
    'Q': 0.095,
    'R': 5.987,
    'S': 6.327,
    'T': 9.056,
    'U': 2.758,
    'V': 0.978,
    'W': 2.360,
    'X': 0.150,
    'Y': 1.974,
    'Z': 0.074
}

plt.bar(letter_frequencies.keys(), letter_frequencies.values())

plt.show()

Now, let's write a function to show the letter frequencies of a particular piece of text, and pass it the plaintext and the ciphertext from earlier:

In [ ]:
from collections import Counter
import base64

def show_frequencies(s: str):
    character_counts = Counter(s.upper())

    plt.bar(list(string.ascii_uppercase), [character_counts.get(char, 0) for char in string.ascii_uppercase])
    plt.show()

show_frequencies(plaintext)
show_frequencies(ciphertext)

The plaintexttext (the first graph) should look very to the average English Language frequency from before! The ciphertext (the second graph) looks different, however: because we are changing each character by the same amount, the graph has been shifted. However, it is easy to tell how far it was shifted just by looking at it! By looking at the very common or very uncommon letters, it is possible to tell how far they have shifted between the two graphs. This number is the supposedly "secret" key! Below is a large block of new ciphertext, and you are not given the secret key. See if you can calculate the key by looking at the frequency graph, without brute-forcing it.

In [ ]:
new_ciphertext = (
    "Nby ZcnhymmAlug Juwyl Nymn cm u gofncmnuay uylivcw wujuwcns nymn nbun jlialymmcpyfs aynm gily xczzcwofn um cn wihnchoym. "
    "Nby 20 gynyl juwyl nymn qcff vyach ch 30 mywihxm. Fchy oj un nby mnuln. Nby lohhcha mjyyx mnulnm mfiqfs, von aynm zumnyl yuwb gchony uznyl sio byul nbcm mcahuf. "
    "U mchafy fuj mbiofx vy wigjfynyx yuwb ncgy sio byul nbcm miohx. Lygygvyl ni loh ch u mnlucabn fchy, uhx loh um fiha um jimmcvfy.")

show_frequencies(new_ciphertext)

Part 2: AES
===========

We can see that the Caesar Cipher is not a useful method of keeping data safe. Instead, we will use the more complex AES cipher. The AES cipher performs a much more complicated operation than simply shifting the character, which we won't go into depth on here, but suffice to say it leaves the output sufficiently scrambled. It also uses a much longer secret: a string of bits, which can be as long as necessary to ensure security.

In [ ]:
from Crypto.Cipher import AES

# Rather than an integer, the secret is now a long string of bytes (length must be a power of 2)
secret = 'GZktMap1FgML5sa2'
aes_cipher = AES.new(secret, AES.MODE_ECB)

# The plaintext must be a multiple of 16 for the cipher to work, so we add some '0's to the end as padding.
# We haven't implemented a way of removing this padding afterwards yet, so you may still see it in the output.
padding = '0' * (16 - (len(plaintext) % 16))

aes_ciphertext = aes_cipher.encrypt(plaintext + padding)

print(aes_ciphertext)

We can see immidiately that the output is far more scrambled than before. There is no resembalance to the original string. We can still decrypt this by reversing the operation, but doing so requires the same secret as before. Using the wrong secret fails to decrypt the ciphertext, and the result is still scrambled!

In [ ]:
print('Original secret')
print('===============')
print(AES.new(key=secret, mode=AES.MODE_ECB).decrypt(aes_ciphertext))
print()
print('Incorrect secret')
print('================')
print(AES.new(key='thisisincorrect!', mode=AES.MODE_ECB).decrypt(aes_ciphertext))

Because the secret is now a long string and not a small integer, brute-forcing the secret is much harder. This secret is 128 bits long, meaning there are **340,282,366,920,938,463,463,374,607,431,768,211,456** possible options! Here is one last piece of ciphertext, see if you can reverse it without knowing the secret in advance.

In [ ]:
new_aes_ciphertext = (
    b"K\xa6\tz\xd5Q\x11\x07\xc9\xa5\x19pX\xbb\xd4\x1e\xcb\x99\xfbow&\xfdy\xcc\x1e\xf1\xc4\x999V\xb8\xb6\x1d\xbf\x7f\xc9\xbe\xf6"
    b"\xd5\x10\xbc\x07|0\xb3O\xd2\x07\x99\xfcr\x05\xd9\xc6\xc8\xae\xdb\xe3\xc7g\xde%\x02\xfb\xfe\xb4\x86\xa3\xa2\x1c\xd0h\xb3\xd6"
    b"oT;\x1fo\xb4\xef\x7f^M\xaa1\x8di\x16o\xceg|\\\xfc\t\xcc\x8f\x10\xc9\xb7\x120g?/\xb0\xc5\xd5\xfc\x1bJI!\x00\xfd\xdd\x94\xe6,"
    b"(\xc2\x0fJ\x96\xdc\xca\xd3^\xc3\xe7\x86\xd5\xe2\x95\xe9\xbfnh\xd0\xce\xf1;")

secret = '????????????????'
aes_cipher = AES.new(secret, AES.MODE_ECB)

print(aes_cipher.decrypt(new_aes_ciphertext))